<div style="position: absolute; top: 0; left: 0; font-family: 'Garamond'; font-size: 16px;">
    <a href="https://github.com/patriciaapenat" style="text-decoration: none; color: inherit;">Patricia Peña Torres</a>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 48px;">
    <strong>Proyecto final, BRFSS-clustering</strong>
</div>

<div align="center" style="font-family: 'Garamond'; font-size: 36px;">
    <strong>5. Aplicación de algoritmos de clustering</strong>
</div>

__________________

<div style="font-family: 'Garamond'; font-size: 14px;">

En este notebook se llava a cabo lo relativo al análisis exploratorio, por la naturaleza de los datos este EDA se ha centrado principalmente en variables demográficas
    
</div>

<div style="font-family: 'Garamond'; font-size: 16px;">
    <strong>Configuración del entorno de trabajo</strong>
</div>

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, Imputer
from scipy.spatial import KDTree
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
import pandas as pd
import random
import os.path
import seaborn as sns
import pickle
import matplotlib.pyplot as plt
import numpy as np
import warnings
import tensorflow as tf
from pyspark import SparkConf, SparkContext
from sklearn.preprocessing import StandardScaler
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
import seaborn as sns
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import PCA
from sklearn.cluster import KMeans as SKLearnKMeans

from functools import reduce

# Ignorar advertencias deprecated
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# configurar gráficos
sns.set(style="whitegrid", context="notebook", palette="mako")

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Configuración de Spark</strong>
</div>

In [3]:
# Si hay un SparkContext existente, debemos cerrarlo antes de crear uno nuevo
if 'sc' in locals() and sc:
    sc.stop()  # Detener el SparkContext anterior si existe

# Configuración de Spark
conf = (
    SparkConf()
    .setAppName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .setMaster("local[2]")  # Modo local con un hilo para ejecución
    .set("spark.driver.host", "127.0.0.1")  # Dirección del host del driver
    .set("spark.executor.heartbeatInterval", "3600s")  # Intervalo de latido del executor
    .set("spark.network.timeout", "7200s")  # Tiempo de espera de la red
    .set("spark.executor.memory", "14g")  # Memoria asignada para cada executor
    .set("spark.driver.memory", "14g")  # Memoria asignada para el driver
)

# Crear un nuevo SparkContext con la configuración especificada
sc = SparkContext(conf=conf)

# Configuración de SparkSession (interfaz de alto nivel para trabajar con datos estructurados en Spark)
spark = (
    SparkSession.builder
    .appName("Proyecto_PatriciaA_Peña")  # Nombre de la aplicación en Spark
    .config("spark.sql.repl.eagerEval.enabled", True)  # Habilitar la evaluación perezosa en Spark SQL REPL
    .config("spark.sql.repl.eagerEval.maxNumRows", 1000)  # Número máximo de filas a mostrar en la evaluación perezosa
    .getOrCreate()  # Obtener la sesión Spark existente o crear una nueva si no existe
) 

<div style="font-family: 'Garamond'; font-size: 14px;">
    <strong>Lectura del archivo</strong>
</div>

In [4]:
df = spark.read.format("csv").option("header", "true").load(r"C:\\Users\\patri\\OneDrive - UAB\\Documentos\\GitHub\\BRFSS-clustering\\datos\\BRFSS_imputated_2022.csv")

In [5]:
# Convertir todas las columnas a tipo numérico
for column_name in df.columns:
    df = df.withColumn(column_name, col(column_name).cast("double"))

# Autoencoder

Un autoencoder es un tipo de red neuronal artificial utilizada en tareas de aprendizaje no supervisado, específicamente en tareas de reducción de dimensionalidad y generación de datos. Su estructura consta de dos partes principales: el codificador (encoder) y el decodificador (decoder). El objetivo principal de un autoencoder es aprender una representación comprimida de los datos de entrada y luego reconstruir los datos originales a partir de esta representación comprimida.

Aquí hay una breve descripción de cada una de las partes de un autoencoder:

1. Codificador (Encoder):
   - La parte del codificador toma los datos de entrada y los transforma en una representación de menor dimensionalidad (también llamada "código" o "embedding").
   - A medida que la red neuronal del codificador reduce la dimensionalidad, está aprendiendo a capturar las características más importantes y relevantes de los datos de entrada.
   - El codificador puede consistir en una o varias capas ocultas, típicamente utilizando funciones de activación como ReLU (Rectified Linear Unit) en cada capa.

2. Decodificador (Decoder):
   - La parte del decodificador toma la representación comprimida del codificador y la expande nuevamente para reconstruir los datos originales.
   - La red del decodificador es esencialmente un espejo inverso del codificador, donde las capas ocultas aumentan gradualmente la dimensionalidad de los datos.
   - El decodificador utiliza una función de activación adecuada en la capa de salida para generar la reconstrucción.

La idea clave detrás de un autoencoder es que la red intenta aprender una representación eficiente de los datos, de modo que la reconstrucción sea lo más cercana posible a los datos originales. El proceso de entrenamiento implica minimizar la diferencia entre los datos de entrada y los datos reconstruidos, lo que fomenta la captura de patrones significativos en los datos.

Los autoencoders tienen diversas aplicaciones, como la reducción de ruido en imágenes, la detección de anomalías, la generación de imágenes sintéticas y la reducción de dimensionalidad para visualización y compresión de datos.

En Keras, puedes implementar un autoencoder utilizando su API de alto nivel, que facilita la construcción y entrenamiento de redes neuronales. Puedes definir un modelo de autoencoder utilizando capas Dense (totalmente conectadas) para el codificador y el decodificador, y luego compilar y entrenar el modelo utilizando datos de entrada y objetivos de reconstrucción.

In [6]:
columnas_features = [col for col in df.columns if col != "etiqueta"]
ensamblador = VectorAssembler(inputCols=columnas_features, outputCol="features")
df_con_features = ensamblador.transform(df).select("features")

In [7]:
# Convertir el DataFrame de Spark a un array NumPy
features_array = np.array(df_con_features.rdd.map(lambda x: x.features.toArray()).collect())

In [8]:
# Definir el autoencoder utilizando TensorFlow
input_dim = len(columnas_features)
encoding_dim = 4  # Dimensión reducida deseada

In [9]:
# Definir la arquitectura del autoencoder
input_layer = tf.keras.layers.Input(shape=(input_dim,))
encoder = tf.keras.layers.Dense(encoding_dim, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l=0.001))(input_layer)
decoder = tf.keras.layers.Dense(input_dim, activation='relu')(encoder)

In [10]:
# Crear el modelo del autoencoder
autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

Un autoencoder es un tipo de red neuronal que consta de dos partes principales: el codificador $(f(x))$ y el decodificador $(g(z))$. El objetivo principal de un autoencoder es aprender una representación eficiente y comprimida de los datos de entrada, de modo que se puedan reconstruir con la menor pérdida de información posible. Para lograr esto, se utiliza una función de pérdida que mide la diferencia entre los datos de entrada originales $(x)$ y los datos reconstruidos $(x')$.

1. Codificador (Encoder):
   - El codificador toma un vector de entrada $x$ y lo mapea a un vector de representación comprimida $z$ a través de una serie de transformaciones lineales y no lineales. En este caso, el codificador utiliza una activación 'sigmoid' y un término de regularización L2 con $l=0.01$, lo que significa que se aplica una función sigmoide a la salida del codificador y se agrega un término de regularización L2 en la función de pérdida para controlar el sobreajuste:
   $$ z = f(x) $$

2. Decodificador (Decoder):
   - El decodificador toma la representación comprimida $z$ y lo mapea de nuevo al espacio de entrada $x'$ tratando de reconstruir $x$ lo más fielmente posible. En este caso, el decodificador utiliza una activación 'relu':
   $$ x' = g(z) $$

3. Función de Pérdida (Loss Function):
   - Para entrenar el autoencoder, se utiliza la función de pérdida 'categorical_crossentropy', que se usa comúnmente para problemas de clasificación multiclase. En este contexto, se utiliza para medir la discrepancia entre las etiquetas asignadas y las salidas del decodificador. La pérdida 'categorical_crossentropy' se utiliza para evaluar qué tan bien se asignan las etiquetas a las representaciones codificadas:
   $$  {categorical_crossentropy}(y, y') = -\sum_{i} y_i \log(y'_i) $$

   - Donde $y$ son las etiquetas reales y $y'$ son las salidas del decodificador.

4. Entrenamiento: **pendiente cambiar**
   - Durante el entrenamiento, el autoencoder busca minimizar la función de pérdida 'categorical_crossentropy', ajustando los parámetros del codificador y el decodificador. Esto se logra utilizando un optimizador Adam con tasas de aprendizaje adaptativas. El término de regularización L2 en el codificador ayuda a controlar el sobreajuste durante el entrenamiento.

El objetivo final es que, después del entrenamiento, el autoencoder aprenda a capturar las características más importantes y relevantes de los datos de entrada en la representación $z$, de modo que la reconstrucción $x'$ sea una versión fiel de $x$, y la pérdida 'categorical_crossentropy' sea mínima. Las representaciones codificadas obtenidas pueden ser útiles en tareas de clasificación o análisis de datos posteriores.

In [11]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.99,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
autoencoder.compile(optimizer=optimizer, loss='mean_squared_error')

In [ ]:
# Entrenar el autoencoder
autoencoder.fit(features_array, features_array, epochs=100, batch_size=32)

Epoch 1/100
11348/11348 [==============================] - 34s 3ms/step - loss: 38959.8320
Epoch 2/100
11348/11348 [==============================] - 35s 3ms/step - loss: 26688.6465
Epoch 3/100
11348/11348 [==============================] - 36s 3ms/step - loss: 17848.7031
Epoch 4/100
11348/11348 [==============================] - 32s 3ms/step - loss: 11697.5020
Epoch 5/100
11348/11348 [==============================] - 32s 3ms/step - loss: 7585.4727
Epoch 6/100
11348/11348 [==============================] - 37s 3ms/step - loss: 5003.9697
Epoch 7/100
11348/11348 [==============================] - 32s 3ms/step - loss: 3556.0151
Epoch 8/100
11348/11348 [==============================] - 33s 3ms/step - loss: 4308.1860
Epoch 9/100
11348/11348 [==============================] - 35s 3ms/step - loss: 3837.8423
Epoch 10/100
11348/11348 [==============================] - 39s 3ms/step - loss: 7018.6738
Epoch 11/100
11348/11348 [==============================] - 34s 3ms/step - loss: 6260.3735
Epoc

In [ ]:
# Obtener las representaciones codificadas de los datos
encoded_features_model = tf.keras.models.Model(inputs=input_layer, outputs=encoder)
encoded_features = encoded_features_model.predict(features_array)

In [ ]:
# Convertir las representaciones codificadas de vuelta a un DataFrame de Spark
encoded_features_rdd = spark.sparkContext.parallelize(encoded_features.tolist())
encoded_features_df = encoded_features_rdd.map(lambda x: (Vectors.dense(x),)).toDF(["encoded_features"])

In [ ]:
# Entrenar el autoencoder
history = autoencoder.fit(features_array, features_array, epochs=100, batch_size=64)

# Imprimir métricas durante el entrenamiento
print("Métricas durante el entrenamiento:")
print(history.history)

In [ ]:
# Visualizar la pérdida durante el entrenamiento
plt.plot(history.history['loss'])
plt.title('Pérdida durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.show()

In [ ]:
# Convertir el DataFrame de Spark a un array NumPy
encoded_features_array = np.array(encoded_features_df.select("encoded_features").rdd.map(lambda x: x.encoded_features.toArray()).collect())

# KMEANS

Qué es Kmeans
Explicación matemática
y del código

In [ ]:
# Crear una lista vacía para almacenar las inercias
cs = []

# Probar diferentes valores de k (número de clusters)
for i in range(1, 20):
    kmeans = SKLearnKMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(encoded_features_array)  # Usar array Numpy de características codificadas

    # Calcular la inercia y añadirla a la lista
    cs.append(kmeans.inertia_)

# Trazar la curva de la inercia en función del número de clusters
plt.plot(range(1, 20), cs, marker='o', linestyle='-', color='blue')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.title('Criterio del Codo')
plt.show()

In [ ]:
# Asegurarse de que las características estén en formato de vector
assembler = VectorAssembler(inputCols=["encoded_features"], outputCol="features")
vectorized_df = assembler.transform(encoded_features_df)

# Estandarizar las características
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=False)
scalerModel = scaler.fit(vectorized_df)
scaled_df = scalerModel.transform(vectorized_df)

# Aplicar PCA
pca = PCA(k=2, inputCol="scaled_features", outputCol="pcaFeatures")
pcaModel = pca.fit(scaled_df)
result = pcaModel.transform(scaled_df)

In [ ]:
# Crear el objeto KMeans
kmeans = KMeans(k=2, featuresCol="pcaFeatures", predictionCol="cluster")

# Entrenar el modelo KMeans
kmeans_model = kmeans.fit(result)

# Obtener las predicciones
predictions = kmeans_model.transform(result)

# Mostrar los resultados
predictions.select('pcaFeatures', 'cluster').show()

In [ ]:
# Coeficiente de silueta
evaluator = ClusteringEvaluator(predictionCol="cluster")
silhouette = evaluator.evaluate(predictions)
print("Coeficiente de silueta:", silhouette)

# Puntuación de Calinski-Harabasz
calinski_harabasz = evaluator.evaluate(predictions, {evaluator.metricName: "silhouette"})
print("Puntuación de Calinski-Harabasz:", calinski_harabasz)

In [ ]:
# Convertir Spark DataFrame a Pandas DataFrame para visualización
predictions_pd = predictions.toPandas()

# Extraer componentes de 'pcaFeatures' en nuevas columnas
predictions_pd['pca_x'] = pcaFeatures_array[:, 0]
predictions_pd['pca_y'] = pcaFeatures_array[:, 1]

# Visualización 2D de los clusters utilizando Seaborn y la paleta "mako"
plt.figure(figsize=(10, 8))
sns.scatterplot(data=predictions_pd, x='pca_x', y='pca_y', hue='cluster', palette='mako', s=50, legend='full')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('2D Visualization of Clusters with Mako Palette')
plt.legend(title='Cluster')
plt.show()